In [1]:
# libraries
import os
import json
import numpy as np
import pandas as pd
from extra_files import helper as hp
from imageio import imwrite, imread
from skimage.transform import resize

In [2]:
# File paths
data_path = '/home/aldo/Documents/data-cic/'
preprocess_path = data_path + 'preprocess_data'

## Training SSDLite300

In [3]:
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TerminateOnNaN, CSVLogger, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras import backend as K
from keras.models import load_model
from math import ceil
import numpy as np
from matplotlib import pyplot as plt

from models.keras_ssdlite320PPN_mobilenetv2 import ssd_300
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization

from ssd_encoder_decoder.ssd_input_encoder import SSDInputEncoder
from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.data_augmentation_chain_original_ssd import SSDDataAugmentation
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms

from extra_files.f1_callback import F1_callback as f1_call
%matplotlib inline

Using TensorFlow backend.
/home/aldo/Documents/ssd/data_generator/object_detection_2d_data_generator.py:43: UserWarning: 'BeautifulSoup' module is missing. The XML-parser will be unavailable.
  warnings.warn("'BeautifulSoup' module is missing. The XML-parser will be unavailable.")


### Parameters (original SSD300 architecture)

In [4]:
## Parameteres needed for ssd_300() and SSDInputEncoder()

img_height = 300 # Height of the model input images
img_width = 300 # Width of the model input images
img_channels = 3 # Number of color channels of the model input images
mean_color = [1., 1., 1.] # The per-channel mean of the images in the dataset. Do not change this value if you're using any of the pre-trained weights.
divide_by_stddev = [127.5, 127.5, 127.5]
swap_channels = False # The color channel order in the original SSD is BGR, so we'll have the model reverse the color channel order of the input images.
n_classes = 1 # Number of positive classes, e.g. 20 for Pascal VOC, 80 for MS COCO
new_scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
scales = new_scales
aspect_ratios = [[1.0, 0.5, 2.0/3.0],
                 [1.0, 0.5, 2.0/3.0, 1.0/3.0, 3.0/4.0],
                 [1.0, 0.5, 2.0/3.0, 1.0/3.0],
                 [1.0, 2.0, 2.0/3.0, 1.0/3.0],
                 [1.0, 2.0, 2.0/3.0, 1.0/3.0],
                 [1.0, 2.0, 2.0/3.0, 1.0/3.0]] # The anchor box aspect ratios used in the original SSD300; the order matters
two_boxes_for_ar1 = True
steps = [16, 30, 60, 100, 150, 300] # The space between two adjacent anchor box center points for each predictor layer.
offsets = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5] # The offsets of the first anchor box center points from the top and left borders of the image as a fraction of the step size for each predictor layer.
clip_boxes = False # Whether or not to clip the anchor boxes to lie entirely within the image boundaries
variances = [0.1, 0.1, 0.2, 0.2] # The variances by which the encoded target coordinates are divided as in the original implementation
normalize_coords = True
size_classifier = 128

## Create new model with SSD weights

In [5]:
# 1: Build the Keras model.

K.clear_session() # Clear previous models from memory.

model = ssd_300(image_size=(img_height, img_width, img_channels),
                n_classes=n_classes,
                alpha=0.35,
                expansion=1,
                mode='training',
                scales=scales,
                aspect_ratios_per_layer=aspect_ratios,
                two_boxes_for_ar1=two_boxes_for_ar1,
                steps=steps,
                offsets=offsets,
                clip_boxes=clip_boxes,
                variances=variances,
                normalize_coords=normalize_coords,
                subtract_mean=mean_color,
                divide_by_stddev=divide_by_stddev,
                swap_channels=swap_channels)

# 3: Instantiate an optimizer and the SSD loss function and compile the model.
#    If you want to follow the original Caffe implementation, use the preset SGD
#    optimizer, otherwise I'd recommend the commented-out Adam optimizer.

adam = Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

model.compile(optimizer=adam, loss=ssd_loss.compute_loss)

In [6]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 300, 300, 3)  0                                            
__________________________________________________________________________________________________
identity_layer (Lambda)         (None, 300, 300, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
input_mean_normalization (Lambd (None, 300, 300, 3)  0           identity_layer[0][0]             
__________________________________________________________________________________________________
input_stddev_normalization (Lam (None, 300, 300, 3)  0           input_mean_normalization[0][0]   
__________________________________________________________________________________________________
model_1 (M

## Data generator for the training

In [7]:
# 1: Instantiate two `DataGenerator` objects: One for training, one for validation.

# Optional: If you have enough memory, consider loading the images into memory for the reasons explained above.

train_dataset = DataGenerator(load_images_into_memory=True, hdf5_dataset_path=None)
val_dataset = DataGenerator(load_images_into_memory=True, hdf5_dataset_path=None)

# 2: Parse the image and label lists for the training and validation datasets.

# TODO: Set the paths to your dataset here.

# Images
# images_dir = data_path + 'udacity_driving_datasets'
images_dir = data_path + 'pascal_dataset'
# images_dir = data_path + 'images'

# Ground truth
# train_labels_filename = preprocess_path + '/udc.csv'
# train_labels_filename = preprocess_path + '/cic_pascal_train.csv'
# val_labels_filename   = preprocess_path + '/cic_pascal_val.csv'
train_labels_filename = preprocess_path + '/pascal_train.csv'
val_labels_filename   = preprocess_path + '/pascal_val.csv'
# train_labels_filename = preprocess_path + '/cic_train.csv'
# val_labels_filename   = preprocess_path + '/cic_val.csv'

train_dataset.parse_csv(images_dir=images_dir,
                        labels_filename=train_labels_filename,
                        input_format=['image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'], # This is the order of the first six columns in the CSV file that contains the labels for your dataset. If your labels are in XML format, maybe the XML parser will be helpful, check the documentation.
                        include_classes='all')

val_dataset.parse_csv(images_dir=images_dir,
                      labels_filename=val_labels_filename,
                      input_format=['image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'],
                      include_classes='all')

# Optional: Convert the dataset into an HDF5 dataset. This will require more disk space, but will
# speed up the training. Doing this is not relevant in case you activated the `load_images_into_memory`
# option in the constructor, because in that cas the images are in memory already anyway. If you don't
# want to create HDF5 datasets, comment out the subsequent two function calls.

# train_dataset.create_hdf5_dataset(file_path='dataset_udacity_traffic_train.h5',
#                                   resize=False,
#                                   variable_image_size=True,
#                                   verbose=True)

# val_dataset.create_hdf5_dataset(file_path='dataset_udacity_traffic_val.h5',
#                                 resize=False,
#                                 variable_image_size=True,
#                                 verbose=True)

# Get the number of samples in the training and validations datasets.
train_dataset_size = train_dataset.get_dataset_size()
val_dataset_size   = val_dataset.get_dataset_size()

print("Number of images in the training dataset:\t{:>6}".format(train_dataset_size))
print("Number of images in the validation dataset:\t{:>6}".format(val_dataset_size))

Loading images into memory: 100%|██████████| 2097/2097 [00:08<00:00, 246.86it/s]
Number of images in the training dataset:	  6469
Number of images in the validation dataset:	  2097


In [8]:
# 3: Set the batch size.
batch_size = 32 # Change the batch size if you like, or if you run into GPU memory issues.

# 4: Set the image transformations for pre-processing and data augmentation options.
# For the training generator:
ssd_data_augmentation = SSDDataAugmentation(img_height=img_height,
                                            img_width=img_width,
                                            background=mean_color)

# For the validation generator:
convert_to_3_channels = ConvertTo3Channels()
resize = Resize(height=img_height, width=img_width)

# 5: Instantiate an encoder that can encode ground truth labels into the format needed by the SSD loss function.
# The encoder constructor needs the spatial dimensions of the model's predictor layers to create the anchor boxes.
# predictor_sizes = [model.get_layer('conv13_mbox_conf').output_shape[1:3],
#                    model.get_layer('conv17_mbox_conf').output_shape[1:3],
#                    model.get_layer('conv18_2_mbox_conf').output_shape[1:3],
#                    model.get_layer('conv19_2_mbox_conf').output_shape[1:3],
#                    model.get_layer('conv20_2_mbox_conf').output_shape[1:3],
#                    model.get_layer('conv21_2_mbox_conf').output_shape[1:3]]

predictor_sizes = [(18, 18), (9, 9), (5, 5), (3, 3), (2, 2), (1, 1)]

ssd_input_encoder = SSDInputEncoder(img_height=img_height,
                                    img_width=img_width,
                                    n_classes=n_classes,
                                    predictor_sizes=predictor_sizes,
                                    scales=scales,
                                    aspect_ratios_per_layer=aspect_ratios,
                                    two_boxes_for_ar1=two_boxes_for_ar1,
                                    steps=steps,
                                    offsets=offsets,
                                    clip_boxes=clip_boxes,
                                    variances=variances,
                                    matching_type='multi',
                                    pos_iou_threshold=0.5,
                                    neg_iou_limit=0.5,
                                    normalize_coords=normalize_coords)

# 6: Create the generator handles that will be passed to Keras' `fit_generator()` function.
train_generator = train_dataset.generate(batch_size=batch_size,
                                         shuffle=True,
                                         transformations=[ssd_data_augmentation],
                                         label_encoder=ssd_input_encoder,
                                         returns={'processed_images',
                                                  'encoded_labels'},
                                         keep_images_without_gt=False)

val_generator = val_dataset.generate(batch_size=batch_size,
                                     shuffle=False,
                                     transformations=[convert_to_3_channels,
                                                      resize],
                                     label_encoder=ssd_input_encoder,
                                     returns={'processed_images',
                                              'encoded_labels'},
                                     keep_images_without_gt=False)

# Get the number of samples in the training and validations datasets.
train_dataset_size = train_dataset.get_dataset_size()
val_dataset_size   = val_dataset.get_dataset_size()

print("Number of images in the training dataset:\t{:>6}".format(train_dataset_size))
print("Number of images in the validation dataset:\t{:>6}".format(val_dataset_size))

Number of images in the training dataset:	  6469
Number of images in the validation dataset:	  2097


## Remaining training parameters

In [9]:
label_val = np.load('../data-cic/preprocess_data/label_val.npy')
val_images_300 = np.load('../data-cic/preprocess_data/images_val_300x300.npy')

In [10]:
# Define model callbacks.

# TODO: Set the filepath under which you want to save the model.
model_checkpoint = ModelCheckpoint(filepath='/home/aldo/Documents/weights/models/new_net/new_share_conv_conf_pascal_a_0.35_e_1_ps_ng_0.5.h5',
                                   monitor='val_loss',
                                   verbose=1,
                                   save_best_only=True,
                                   save_weights_only=False,
                                   mode='auto',
                                   period=1)

csv_logger = CSVLogger(filename=data_path + 'history/new_net/new_share_conv_conf_pascal_a_0.35_e_1_ps_ng_0.5.csv',
                       separator=',',
                       append=True)

reduce_learning_rate = ReduceLROnPlateau(monitor='val_loss',
                                         factor=0.5,
                                         patience=10,
                                         verbose=1,
                                         min_delta=0.001,
                                         cooldown=0,
                                         min_lr=0.000001)

tbCallBack = TensorBoard(log_dir=data_path + 'history/new_net/new_share_conv_conf_pascal_a_0.35_e_1_ps_ng_0.5', 
                                   histogram_freq=0, 
                                   write_graph=False, 
                                   write_images=False,
                                   update_freq='epoch')

f1_callback = f1_call(0.20, 
                      0.45, 
                      200, 
                      normalize_coords, 
                      img_height, 
                      img_width, 
                      val_images_300, 
                      label_val, (1, 1977, 14),
                      data_path + 'history/new_net/new_share_conv_conf_pascal_f1_a_0.35_e_1_ps_ng_0.5.csv',
                      '/home/aldo/Documents/weights/models/new_net/new_share_conv_conf_pascal_f1_a_0.35_e_1_ps_ng_0.5.h5')


callbacks = [model_checkpoint,
             csv_logger,
             reduce_learning_rate,
             f1_callback,
             tbCallBack]

In [11]:
# If you're resuming a previous training, set `initial_epoch` and `final_epoch` accordingly.
initial_epoch   = 0
final_epoch     = 120
steps_per_epoch = 400

history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=steps_per_epoch,
                              epochs=final_epoch,
                              callbacks=callbacks,
                              validation_data=val_generator,
                              validation_steps=ceil(val_dataset_size/batch_size),
                              initial_epoch=initial_epoch)

Epoch 1/120
400/400 [==============================] - 225s 561ms/step - loss: 10.1451 - val_loss: 9.2928

Epoch 00001: val_loss improved from inf to 9.29276, saving model to /home/aldo/Documents/weights/models/new_net/new_share_conv_conf_pascal_a_0.35_e_1_ps_ng_0.5.h5
F1 score: 0.11225141226327574
Improve F1 score from -inf to 0.11225141226327574
Epoch 2/120
400/400 [==============================] - 215s 537ms/step - loss: 4.6065 - val_loss: 11.3443

Epoch 00002: val_loss did not improve from 9.29276
F1 score: 0.1002673701688534
Epoch 3/120
400/400 [==============================] - 214s 534ms/step - loss: 4.3191 - val_loss: 6.9097

Epoch 00003: val_loss improved from 9.29276 to 6.90971, saving model to /home/aldo/Documents/weights/models/new_net/new_share_conv_conf_pascal_a_0.35_e_1_ps_ng_0.5.h5
F1 score: 0.10305863993195208
Epoch 4/120
400/400 [==============================] - 216s 539ms/step - loss: 4.2021 - val_loss: 7.0112

Epoch 00004: val_loss did not improve from 6.90971
F1 

400/400 [==============================] - 212s 529ms/step - loss: 3.3225 - val_loss: 6.1866

Epoch 00037: val_loss did not improve from 5.34718
F1 score: 0.3708463420495935
Epoch 38/120
400/400 [==============================] - 210s 525ms/step - loss: 3.3283 - val_loss: 6.2437

Epoch 00038: val_loss did not improve from 5.34718

Epoch 00038: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
F1 score: 0.38247076503094224
Epoch 39/120
400/400 [==============================] - 214s 535ms/step - loss: 3.2568 - val_loss: 6.0540

Epoch 00039: val_loss did not improve from 5.34718
F1 score: 0.39090156621732086
Epoch 40/120
400/400 [==============================] - 213s 532ms/step - loss: 3.2688 - val_loss: 6.0754

Epoch 00040: val_loss did not improve from 5.34718
F1 score: 0.4008625369256666
Epoch 41/120
400/400 [==============================] - 213s 532ms/step - loss: 3.2735 - val_loss: 6.0695

Epoch 00041: val_loss did not improve from 5.34718
F1 score: 0.389799338005

400/400 [==============================] - 210s 525ms/step - loss: 3.1661 - val_loss: 6.1549

Epoch 00078: val_loss did not improve from 5.34718

Epoch 00078: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.
F1 score: 0.408641886604282
Epoch 79/120
400/400 [==============================] - 213s 532ms/step - loss: 3.1691 - val_loss: 6.1290

Epoch 00079: val_loss did not improve from 5.34718
F1 score: 0.4074305213874762
Epoch 80/120
400/400 [==============================] - 212s 531ms/step - loss: 3.1470 - val_loss: 7.8839

Epoch 00080: val_loss did not improve from 5.34718
F1 score: 0.4046986520475543
Epoch 81/120
400/400 [==============================] - 213s 533ms/step - loss: 3.1630 - val_loss: 6.3466

Epoch 00081: val_loss did not improve from 5.34718
F1 score: 0.4020465576255424
Epoch 82/120
400/400 [==============================] - 213s 532ms/step - loss: 3.1552 - val_loss: 6.3442

Epoch 00082: val_loss did not improve from 5.34718
F1 score: 0.409997699022100


Epoch 00119: val_loss did not improve from 5.24762
F1 score: 0.39456733114849246
Epoch 120/120
400/400 [==============================] - 212s 530ms/step - loss: 3.1387 - val_loss: 6.3465

Epoch 00120: val_loss did not improve from 5.24762
F1 score: 0.414381313873212


In [12]:
# Reset learning rate to 0.001
adam = Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)
model.compile(optimizer=adam, loss=ssd_loss.compute_loss)

In [13]:
# 1: Instantiate two `DataGenerator` objects: One for training, one for validation.

# Optional: If you have enough memory, consider loading the images into memory for the reasons explained above.

train_dataset = DataGenerator(load_images_into_memory=True, hdf5_dataset_path=None)
val_dataset = DataGenerator(load_images_into_memory=True, hdf5_dataset_path=None)

# 2: Parse the image and label lists for the training and validation datasets.

# TODO: Set the paths to your dataset here.

# Images
# images_dir = data_path + 'udacity_driving_datasets'
# images_dir = data_path + 'pascal_dataset'
images_dir = data_path + 'images'

# Ground truth
# train_labels_filename = preprocess_path + '/udc.csv'
# train_labels_filename = preprocess_path + '/cic_pascal_train.csv'
# val_labels_filename   = preprocess_path + '/cic_pascal_val.csv'
# train_labels_filename = preprocess_path + '/pascal_train.csv'
# val_labels_filename   = preprocess_path + '/pascal_val.csv'
train_labels_filename = preprocess_path + '/cic_train.csv'
val_labels_filename   = preprocess_path + '/cic_val.csv'

train_dataset.parse_csv(images_dir=images_dir,
                        labels_filename=train_labels_filename,
                        input_format=['image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'], # This is the order of the first six columns in the CSV file that contains the labels for your dataset. If your labels are in XML format, maybe the XML parser will be helpful, check the documentation.
                        include_classes='all')

val_dataset.parse_csv(images_dir=images_dir,
                      labels_filename=val_labels_filename,
                      input_format=['image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'],
                      include_classes='all')

# Optional: Convert the dataset into an HDF5 dataset. This will require more disk space, but will
# speed up the training. Doing this is not relevant in case you activated the `load_images_into_memory`
# option in the constructor, because in that cas the images are in memory already anyway. If you don't
# want to create HDF5 datasets, comment out the subsequent two function calls.

# train_dataset.create_hdf5_dataset(file_path='dataset_udacity_traffic_train.h5',
#                                   resize=False,
#                                   variable_image_size=True,
#                                   verbose=True)

# val_dataset.create_hdf5_dataset(file_path='dataset_udacity_traffic_val.h5',
#                                 resize=False,
#                                 variable_image_size=True,
#                                 verbose=True)

# Get the number of samples in the training and validations datasets.
train_dataset_size = train_dataset.get_dataset_size()
val_dataset_size   = val_dataset.get_dataset_size()

print("Number of images in the training dataset:\t{:>6}".format(train_dataset_size))
print("Number of images in the validation dataset:\t{:>6}".format(val_dataset_size))

Loading images into memory: 100%|██████████| 45/45 [00:01<00:00, 27.76it/s]
Number of images in the training dataset:	   210
Number of images in the validation dataset:	    45


In [14]:
# 3: Set the batch size.
batch_size = 32 # Change the batch size if you like, or if you run into GPU memory issues.

# 4: Set the image transformations for pre-processing and data augmentation options.
# For the training generator:
ssd_data_augmentation = SSDDataAugmentation(img_height=img_height,
                                            img_width=img_width,
                                            background=mean_color)

# For the validation generator:
convert_to_3_channels = ConvertTo3Channels()
resize = Resize(height=img_height, width=img_width)

# 5: Instantiate an encoder that can encode ground truth labels into the format needed by the SSD loss function.
# The encoder constructor needs the spatial dimensions of the model's predictor layers to create the anchor boxes.
# predictor_sizes = [model.get_layer('conv13_mbox_conf').output_shape[1:3],
#                    model.get_layer('conv17_mbox_conf').output_shape[1:3],
#                    model.get_layer('conv18_2_mbox_conf').output_shape[1:3],
#                    model.get_layer('conv19_2_mbox_conf').output_shape[1:3],
#                    model.get_layer('conv20_2_mbox_conf').output_shape[1:3],
#                    model.get_layer('conv21_2_mbox_conf').output_shape[1:3]]

predictor_sizes = [(18, 18), (9, 9), (5, 5), (3, 3), (2, 2), (1, 1)]

ssd_input_encoder = SSDInputEncoder(img_height=img_height,
                                    img_width=img_width,
                                    n_classes=n_classes,
                                    predictor_sizes=predictor_sizes,
                                    scales=scales,
                                    aspect_ratios_per_layer=aspect_ratios,
                                    two_boxes_for_ar1=two_boxes_for_ar1,
                                    steps=steps,
                                    offsets=offsets,
                                    clip_boxes=clip_boxes,
                                    variances=variances,
                                    matching_type='multi',
                                    pos_iou_threshold=0.5,
                                    neg_iou_limit=0.5,
                                    normalize_coords=normalize_coords)

# 6: Create the generator handles that will be passed to Keras' `fit_generator()` function.
train_generator = train_dataset.generate(batch_size=batch_size,
                                         shuffle=True,
                                         transformations=[ssd_data_augmentation],
                                         label_encoder=ssd_input_encoder,
                                         returns={'processed_images',
                                                  'encoded_labels'},
                                         keep_images_without_gt=False)

val_generator = val_dataset.generate(batch_size=batch_size,
                                     shuffle=False,
                                     transformations=[convert_to_3_channels,
                                                      resize],
                                     label_encoder=ssd_input_encoder,
                                     returns={'processed_images',
                                              'encoded_labels'},
                                     keep_images_without_gt=False)

# Get the number of samples in the training and validations datasets.
train_dataset_size = train_dataset.get_dataset_size()
val_dataset_size   = val_dataset.get_dataset_size()

print("Number of images in the training dataset:\t{:>6}".format(train_dataset_size))
print("Number of images in the validation dataset:\t{:>6}".format(val_dataset_size))

Number of images in the training dataset:	   210
Number of images in the validation dataset:	    45


In [15]:
# Define model callbacks.

# TODO: Set the filepath under which you want to save the model.
model_checkpoint = ModelCheckpoint(filepath='/home/aldo/Documents/weights/models/new_net/new_share_conv_conf_cic_a_0.35_e_1_ps_ng_0.5.h5',
                                   monitor='val_loss',
                                   verbose=1,
                                   save_best_only=True,
                                   save_weights_only=False,
                                   mode='auto',
                                   period=1)

csv_logger = CSVLogger(filename=data_path + 'history/new_net/new_share_conv_conf_cic_a_0.35_e_1_ps_ng_0.5.csv',
                       separator=',',
                       append=True)

reduce_learning_rate = ReduceLROnPlateau(monitor='val_loss',
                                         factor=0.5,
                                         patience=10,
                                         verbose=1,
                                         min_delta=0.001,
                                         cooldown=0,
                                         min_lr=0.000001)

tbCallBack = TensorBoard(log_dir=data_path + 'history/new_net/new_share_conv_conf_cic_a_0.35_e_1_ps_ng_0.5', 
                                   histogram_freq=0, 
                                   write_graph=False, 
                                   write_images=False,
                                   update_freq='epoch')

f1_callback = f1_call(0.20, 
                      0.45, 
                      200, 
                      normalize_coords, 
                      img_height, 
                      img_width, 
                      val_images_300, 
                      label_val, (1, 1977, 14),
                      data_path + 'history/new_net/new_share_conv_conf_cic_f1_a_0.35_e_1_ps_ng_0.5.csv',
                      '/home/aldo/Documents/weights/models/new_net/new_share_conv_conf_cic_f1_a_0.35_e_1_ps_ng_0.5.h5')


callbacks = [model_checkpoint,
             csv_logger,
             reduce_learning_rate,
             f1_callback,
             tbCallBack]

In [16]:
# If you're resuming a previous training, set `initial_epoch` and `final_epoch` accordingly.
initial_epoch   = 0
final_epoch     = 100
steps_per_epoch = 15

history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=steps_per_epoch,
                              epochs=final_epoch,
                              callbacks=callbacks,
                              validation_data=val_generator,
                              validation_steps=ceil(val_dataset_size/batch_size),
                              initial_epoch=initial_epoch)

Epoch 1/100
15/15 [==============================] - 64s 4s/step - loss: 3.6248 - val_loss: 4.6458

Epoch 00001: val_loss improved from inf to 4.64579, saving model to /home/aldo/Documents/weights/models/new_net/new_share_conv_conf_cic_a_0.35_e_1_ps_ng_0.5.h5
F1 score: 0.4179762281219451
Improve F1 score from -inf to 0.4179762281219451
Epoch 2/100
15/15 [==============================] - 58s 4s/step - loss: 3.6241 - val_loss: 4.6493

Epoch 00002: val_loss did not improve from 4.64579
F1 score: 0.4239959715403958
Improve F1 score from 0.4179762281219451 to 0.4239959715403958
Epoch 3/100
15/15 [==============================] - 63s 4s/step - loss: 3.3807 - val_loss: 4.6450

Epoch 00003: val_loss improved from 4.64579 to 4.64495, saving model to /home/aldo/Documents/weights/models/new_net/new_share_conv_conf_cic_a_0.35_e_1_ps_ng_0.5.h5
F1 score: 0.4280749972041522
Improve F1 score from 0.4239959715403958 to 0.4280749972041522
Epoch 4/100
15/15 [==============================] - 63s 4s/ste

15/15 [==============================] - 65s 4s/step - loss: 2.7848 - val_loss: 4.0190

Epoch 00041: val_loss improved from 4.55057 to 4.01900, saving model to /home/aldo/Documents/weights/models/new_net/new_share_conv_conf_cic_a_0.35_e_1_ps_ng_0.5.h5
F1 score: 0.46291312645017246
Epoch 42/100
15/15 [==============================] - 62s 4s/step - loss: 2.7563 - val_loss: 4.5808

Epoch 00042: val_loss did not improve from 4.01900
F1 score: 0.46505736835551076
Improve F1 score from 0.4643783516294074 to 0.46505736835551076
Epoch 43/100
15/15 [==============================] - 63s 4s/step - loss: 2.6875 - val_loss: 4.5197

Epoch 00043: val_loss did not improve from 4.01900
F1 score: 0.4765261180424623
Improve F1 score from 0.46505736835551076 to 0.4765261180424623
Epoch 44/100
15/15 [==============================] - 63s 4s/step - loss: 2.7508 - val_loss: 4.4435

Epoch 00044: val_loss did not improve from 4.01900
F1 score: 0.474585639651291
Epoch 45/100
15/15 [===========================

Epoch 80/100
15/15 [==============================] - 62s 4s/step - loss: 2.7650 - val_loss: 4.1644

Epoch 00080: val_loss did not improve from 4.01900
F1 score: 0.513686556931954
Improve F1 score from 0.5063893362047655 to 0.513686556931954
Epoch 81/100
15/15 [==============================] - 65s 4s/step - loss: 2.6128 - val_loss: 4.1603

Epoch 00081: val_loss did not improve from 4.01900

Epoch 00081: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.
F1 score: 0.5176043289367916
Improve F1 score from 0.513686556931954 to 0.5176043289367916
Epoch 82/100
15/15 [==============================] - 60s 4s/step - loss: 2.6536 - val_loss: 4.2225

Epoch 00082: val_loss did not improve from 4.01900
F1 score: 0.5180819660160568
Improve F1 score from 0.5176043289367916 to 0.5180819660160568
Epoch 83/100
15/15 [==============================] - 63s 4s/step - loss: 2.6202 - val_loss: 4.2184

Epoch 00083: val_loss did not improve from 4.01900
F1 score: 0.5184470324817241
Improve F